In [1]:
import torch
from torch.utils.data import DataLoader
from dataset.dataset import get_cdiscount_dataset
from model.model import assemble_model
from trainer.trainer import get_trainer

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# redirect print to file
# import sys
# sys.stdout = open("PyTorch-resnet34-log.txt", "w")

In [2]:
# parameters
config = {
    'train_batch_size': 256, 'val_batch_size': 256,
    'arch': 'resnet34',
    'optimizer': 'Adam', 'learning_rate': 1e-4, 'decay_lr_freq': 1e4, 'weight_decay': 5e-4,
    'resume': None,
    'start_epoch': 0, 'epochs': 10,
    'print_freq': 10, 'validate_freq': 3e4, 'save_freq': 1e3,
    'best_val_prec1': 0
}

In [ ]:
import torchvision.models as models

# get dataset
print('getting dataset...')
train_dataset = get_cdiscount_dataset(offsets_csv="train_offsets.csv",
                                     images_csv="train_images.csv",
                                     with_label=True,
                                     resize=256)
val_dataset = get_cdiscount_dataset(offsets_csv="train_offsets.csv",
                                     images_csv="val_images.csv",
                                     with_label=True,
                                     resize=256)

# get data loader
print('getting data loader...')
train_dataloader = DataLoader(train_dataset, batch_size=config['train_batch_size'], shuffle=True, num_workers=6)
val_dataloader = DataLoader(val_dataset, batch_size=config['val_batch_size'], shuffle=True, num_workers=6)

# define model
print("=> using pre-trained model '{}'".format(config['arch']))
model = models.__dict__[config['arch']](pretrained=True)
model = assemble_model(model, -1, 512, 5270)
model = torch.nn.DataParallel(model).cuda()

# define loss function (criterion) and optimizer
criterion = torch.nn.CrossEntropyLoss().cuda()

# get trainer
Trainer = get_trainer(train_dataloader, val_dataloader, model, criterion, config)

# Run!
Trainer.run()

getting dataset...
getting data loader...
=> using pre-trained model 'resnet34'
start training
Epoch: [0][0/38676]	Time 25.487 (25.487)	Data 9.610 (9.610)	Loss 8.8593 (8.8593)	Prec@1 0.000 (0.000)	Prec@5 0.000 (0.000)
Epoch: [0][10/38676]	Time 3.819 (5.682)	Data 0.000 (0.875)	Loss 8.0490 (8.4861)	Prec@1 3.125 (1.065)	Prec@5 6.641 (2.060)
Epoch: [0][20/38676]	Time 3.797 (4.798)	Data 0.002 (0.459)	Loss 7.4226 (8.1454)	Prec@1 7.422 (3.497)	Prec@5 13.672 (6.362)
Epoch: [0][30/38676]	Time 3.840 (4.486)	Data 0.002 (0.312)	Loss 6.8526 (7.8094)	Prec@1 8.594 (5.595)	Prec@5 18.359 (10.068)
Epoch: [0][40/38676]	Time 3.803 (4.330)	Data 0.002 (0.237)	Loss 6.4529 (7.5290)	Prec@1 12.500 (7.269)	Prec@5 21.875 (12.929)
Epoch: [0][50/38676]	Time 3.774 (4.230)	Data 0.002 (0.191)	Loss 5.9713 (7.2841)	Prec@1 17.578 (8.938)	Prec@5 32.422 (15.862)
Epoch: [0][60/38676]	Time 3.808 (4.162)	Data 0.002 (0.160)	Loss 6.3411 (7.0878)	Prec@1 15.625 (10.220)	Prec@5 26.172 (17.918)
Epoch: [0][70/38676]	Time 3.816 (4.11